In [1]:
import os
import json
import csv
import pandas as pd
import numpy as np
import math
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
# results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_11_13_2023_merged_v10.csv"
# results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_11_13_2023_merged_v11.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_11_13_2023_merged_CHAOS_v12.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_11_13_2023_merged_LVSC_v12.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_2_20_2024_merged_v13.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_11_16_2023_merged_edited1.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_5_31_2024_lab_v17.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_6_5_2024_lab_v17.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_6_13_2024_lab_v17.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_6_20_2024_lab_v17.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_5_20_2024_lab_v15.csv"
#results_csv_file = r"/Users/admin/Documents/research/active learning/results/results_8_1_2024_lab.csv"
#results_csv_file = r"/Users/admin/Documents/research/active learning/results/results_8_1_2024_lab_v15.csv"
results_csv_file = r"/Users/admin/Documents/research/active learning/results/results_8_2_2024_lab_v18.csv"

In [3]:
exp_df = pd.read_csv(results_csv_file)  
ignore_string_in_name_lst = ["CHAOS"]
include_string_in_name_lst = ["sup"]

In [4]:
included_exp_names = exp_df['exp'].tolist()

In [5]:
def get_base_exp_name(exp_name):
    exp_field = "exp"
    exp_field_len_w_index = len(exp_field)+1
    exp_field_index = exp_name.find(exp_field)
    exp_name = exp_name[:exp_field_index]+exp_name[exp_field_index+exp_field_len_w_index:]
    return exp_name

In [6]:
exp_groups = dict()

for exp_name in included_exp_names:
    base_exp_name = get_base_exp_name(exp_name)
    if base_exp_name in exp_groups:
        exp_groups[base_exp_name] += [exp_name]
    else:
        exp_groups[base_exp_name] = [exp_name]

In [7]:
exp_length = 5
num_rounds = 10
round_strings =  ["round" + str(round_num) + "_test" for round_num in range(num_rounds)]
base_exp_names = list(exp_groups.keys())

In [8]:
results_dict = {}

for base_exp_name, exp_group in exp_groups.items():
    continue_flag = False
    for ignore_string_in_name in ignore_string_in_name_lst:
        if ignore_string_in_name in base_exp_name:
            continue_flag = True
            break
    for include_string_in_name in include_string_in_name_lst:
        if include_string_in_name not in base_exp_name:
            print("rejected, ", base_exp_name)
            continue_flag = True
            break
    if continue_flag:
        continue
    exp_dict = {}
    for i, run in enumerate(exp_group):
        sub_df = exp_df.loc[exp_df['exp'] == run]
        for round_string in round_strings:
            val = sub_df[round_string].iloc[0]
            if not math.isnan(val):
                if round_string not in exp_dict:
                    exp_dict[round_string] = [float(val)]
                else:
                    exp_dict[round_string] += [float(val)]
    results_dict[base_exp_name] = exp_dict
    print("accepted ", base_exp_name)

accepted  LVMMED_sup__coreset_v18
accepted  LVMMED_sup__coreset_xt0_pos_loss1_wt033_pos_loss2_wt033_pos_loss3_wt033_use_slice_pos_use_phase_use_patient_v18
accepted  LVMMED_sup__random_v18
accepted  LVMMED_sup__coreset_pos_loss1_wt035_pos_loss2_wt005_use_phase_use_patient_v18


In [9]:
index = pd.Index(list(results_dict.keys()), name="exp_name")
round_counts_strings = [[round_string, f"round{i}_count"] for i, round_string in enumerate(round_strings)]
mean_strings = round_strings[1:5]
#mean_strings = round_strings[0:4]
mean_string_col = "mean_test"
df = pd.DataFrame(index=index, 
                  columns=[mean_string_col] + sum(round_counts_strings, []))

In [10]:
for base_exp_name, results in results_dict.items():
    sub_df = df.loc[base_exp_name]
    results = results_dict[base_exp_name]
    mean_result_list = []
    for round_string, count_string in round_counts_strings:
        if round_string in results:
            round_results = results[round_string]
            num_results = len(round_results)
            mean_result = np.mean(round_results)
            std_results = np.std(round_results, ddof=1)
            mean_string = '%.3f' % mean_result
            std_string = '%.3f'% std_results
            sub_df[round_string] = f"{mean_string}+/-{std_string}"
            sub_df[count_string] = num_results
            if round_string in mean_strings:
                mean_result_list.append(mean_result)
        sub_df[mean_string_col] = np.sum(mean_result_list)/len(mean_result_list)

/var/folders/d4/_mqx502j3f98n_z9jm_d2g900000gq/T/ipykernel_85153/2566401622.py:17: RuntimeWarning: invalid value encountered in scalar divide
  sub_df[mean_string_col] = np.sum(mean_result_list)/len(mean_result_list)


In [11]:
df
df.sort_values(by=[mean_string_col], ascending=False)

,mean_test,round0_test,round0_count,round1_test,round1_count,round2_test,round2_count,round3_test,round3_count,round4_test,round4_count,round5_test,round5_count,round6_test,round6_count,round7_test,round7_count,round8_test,round8_count,round9_test,round9_count
exp_name,,,,,,,,,,,,,,,,,,,,,
LVMMED_sup__random_v18,0.825722,0.619+/-0.059,5,0.778+/-0.016,5,0.819+/-0.012,5,0.850+/-0.008,5,0.856+/-0.007,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LVMMED_sup__coreset_xt0_pos_loss1_wt033_pos_loss2_wt033_pos_loss3_wt033_use_slice_pos_use_phase_use_patient_v18,0.824856,0.635+/-0.039,5,0.779+/-0.009,5,0.822+/-0.008,5,0.844+/-0.008,5,0.855+/-0.005,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LVMMED_sup__coreset_pos_loss1_wt035_pos_loss2_wt005_use_phase_use_patient_v18,0.823997,0.533+/-0.069,5,0.780+/-0.017,5,0.809+/-0.029,5,0.845+/-0.009,5,0.862+/-0.005,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LVMMED_sup__coreset_v18,0.816893,0.566+/-0.073,5,0.782+/-0.021,5,0.814+/-0.007,5,0.833+/-0.013,5,0.840+/-0.010,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
